In [13]:
#
# classifer notebook
#
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Lambda, Compose
from LandmarkDataset import LandmarkDataset

#
# MODEL
#

class NN(torch.nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)
        #self.fc2 = torch.nn.Linear(hidden_size, 48)
        #self.fc3 = torch.nn.Linear(48, num_classes)
        self.dropout = torch.nn.Dropout(p=0.2)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)
        
        out = self.fc2(out)
        #out = self.relu(out)
        #out = self.dropout(out)
        
        #out = self.fc3(out)
        
        return out

    
#
# PARAMS
#

batch_size = 4

input_size = 2 * (21 * 3) + 12 #138
hidden_size = 96
num_classes = 7   # LOOKIE!

learning_rate = .01
num_epochs = 500


#
# INITIAL DATA
#

transformations = Compose([
    Lambda(lambda x: torch.tensor(x.values).float())
])

dataset = LandmarkDataset("/home/jovyan/train/data/landmarks_reset_class.csv",
                              transform=transformations)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#
# INIT MODELS, LOSS FN, GRAD
#

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NN(input_size, hidden_size, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

#
# TRAIN
#

running_loss_epoch = 0
for epoch in range(num_epochs):
    for batch_idx, (labels, landmarks) in enumerate(dataloader):  

        # zero out accumulated gradients
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(landmarks)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        # NB: len(dataloader) is num of batches
        running_loss_epoch += loss.item()
        if ((epoch+1) % 10 == 0) and ((batch_idx+1) % len(dataloader) == 0):
            print ('Epoch [{}/{}], Loss: {:.4f}, Running Loss (per N epoch): {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item(), running_loss_epoch))
            running_loss_epoch = 0


Epoch [10/500], Loss: 0.8144, Running Loss (per N epoch): 1013.7775
Epoch [20/500], Loss: 2.1952, Running Loss (per N epoch): 720.5036
Epoch [30/500], Loss: 1.0533, Running Loss (per N epoch): 660.3523
Epoch [40/500], Loss: 2.4847, Running Loss (per N epoch): 591.4588
Epoch [50/500], Loss: 0.0000, Running Loss (per N epoch): 555.5151
Epoch [60/500], Loss: 0.0078, Running Loss (per N epoch): 515.3041
Epoch [70/500], Loss: 0.0351, Running Loss (per N epoch): 533.1130
Epoch [80/500], Loss: 0.2971, Running Loss (per N epoch): 498.2771
Epoch [90/500], Loss: 0.5499, Running Loss (per N epoch): 443.5748
Epoch [100/500], Loss: 0.0433, Running Loss (per N epoch): 415.2145
Epoch [110/500], Loss: 0.0702, Running Loss (per N epoch): 417.9824
Epoch [120/500], Loss: 0.0925, Running Loss (per N epoch): 410.5061
Epoch [130/500], Loss: 0.3240, Running Loss (per N epoch): 419.4872
Epoch [140/500], Loss: 1.0959, Running Loss (per N epoch): 436.0987
Epoch [150/500], Loss: 0.3080, Running Loss (per N epoch

In [14]:
#
# EXPORT
#

dummy_input = torch.zeros(input_size)
#model.load_state_dict(torch.load('./model_overfit.pt'))
torch.onnx.export(model, dummy_input, 'onnx_model.onnx', export_params=True,
                  input_names = ['landmarks'], output_names = ['class'], verbose=True)

#import onnx
#onnx_model = onnx.load("./onnx_model.onnx")
#onnx.checker.check_model(onnx_model)


Exported graph: graph(%landmarks : Float(138, strides=[1], requires_grad=0, device=cpu),
      %fc1.bias : Float(96, strides=[1], requires_grad=1, device=cpu),
      %fc2.bias : Float(7, strides=[1], requires_grad=1, device=cpu),
      %onnx::MatMul_12 : Float(138, 96, strides=[1, 138], requires_grad=0, device=cpu),
      %onnx::MatMul_13 : Float(96, 7, strides=[1, 96], requires_grad=0, device=cpu)):
  %onnx::Add_6 : Float(96, strides=[1], device=cpu) = onnx::MatMul[onnx_name="MatMul_0"](%landmarks, %onnx::MatMul_12) # /opt/conda/lib/python3.10/site-packages/torch/nn/modules/linear.py:114:0
  %input : Float(96, strides=[1], requires_grad=1, device=cpu) = onnx::Add[onnx_name="Add_1"](%fc1.bias, %onnx::Add_6) # /opt/conda/lib/python3.10/site-packages/torch/nn/modules/linear.py:114:0
  %input.3 : Float(96, strides=[1], requires_grad=1, device=cpu) = onnx::Relu[onnx_name="Relu_2"](%input) # /opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1453:0
  %onnx::Add_10 : Float(7, str

In [15]:
#
# EVAL
#
torch.set_printoptions(precision=4, sci_mode=False)
softmax = torch.nn.Softmax(dim=1)
accuracy = 0
count = 0
with torch.no_grad():
    
    for batch_idx, (labels, landmarks) in enumerate(dataloader):  
        out = model(landmarks)
        prob = softmax(out.data)     #setup for threshold or 'garbage' class
        _, klass = torch.max(out.data, 1)
        
        print(klass, labels, klass==labels)
        print(prob)
        print("-----")
        
        accuracy += (klass == labels).sum().item()
        count += len(labels)
        

print('--------')
print("Accuracy {}/{} : {:.4f}".format(accuracy, count, accuracy/count))


tensor([6, 0, 0, 6]) tensor([6, 0, 0, 6]) tensor([True, True, True, True])
tensor([[    0.0000,     0.0000,     0.0000,     0.0000,     0.0192,     0.0005,
             0.9803],
        [    0.8771,     0.0083,     0.0002,     0.0941,     0.0204,     0.0000,
             0.0000],
        [    0.8778,     0.0163,     0.0005,     0.0824,     0.0230,     0.0000,
             0.0000],
        [    0.0000,     0.0000,     0.0000,     0.0000,     0.0273,     0.0006,
             0.9721]])
-----
tensor([0, 2, 6, 2]) tensor([3, 2, 4, 2]) tensor([False,  True, False,  True])
tensor([[    0.9201,     0.0000,     0.0000,     0.0799,     0.0000,     0.0000,
             0.0000],
        [    0.0843,     0.3740,     0.4192,     0.0027,     0.0058,     0.0000,
             0.1141],
        [    0.0000,     0.0000,     0.0000,     0.0003,     0.2154,     0.0003,
             0.7841],
        [    0.0843,     0.3740,     0.4192,     0.0027,     0.0058,     0.0000,
             0.1141]])
-----
tensor([

In [16]:
print("Done")

Done
